<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/6_simple-model/gp-simple-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단하게 모델 만들어보기

map feature는 무시하고, agent만 고렿

In [3]:
# GCS에 접근하기 위해 인증

from google.colab import auth
auth.authenticate_user()

In [4]:
!gsutil du -ch -s "gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/"

645.92 GiB   gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario
645.92 GiB   total


In [8]:
!gsutil ls -l "gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/training/*-000*-of-*" | wc -l

101


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
GDRIVE_WOD_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed"
!mkdir {GDRIVE_WOD_PATH}/scenario/training
!ls -al {GDRIVE_WOD_PATH}/scenario

total 16
drwx------ 2 root root 4096 Apr 25 14:43 testing
drwx------ 2 root root 4096 Apr 26 12:46 training
drwx------ 2 root root 4096 Apr 25 15:24 training_20s
drwx------ 2 root root 4096 Apr 14 03:29 validation


In [14]:
!gsutil -m cp -r \
  "gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/training/*-001*-of-*" \
  {GDRIVE_WOD_PATH}/scenario/training

Copying gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/training/training.tfrecord-00102-of-01000...
Copying gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/training/training.tfrecord-00100-of-01000...
Copying gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/training/training.tfrecord-00101-of-01000...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/training/training.tfrecord-00105-of-01000...
Copying gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/training/training.tfrecord-00103-of-01000...
Copying gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/training/training.tfrecord-00109-of-01000...
Copying gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/scenario/tr

## 1. 환경세팅

In [31]:
from google.colab import drive
drive.mount('/content/drive')

# 구글 드라이브 위치. 마운트 잘 됐는지 확인.
GDRIVE_WOD_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed"
TRAIN_SET_PATH  = GDRIVE_WOD_PATH + "/scenario/training"
VALID_SET_PATH  = GDRIVE_WOD_PATH + "/scenario/validation"

# 마운트 확인
!ls -al {VALIDSET_PATH} | wc -l

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ls: cannot access '{VALIDSET_PATH}': No such file or directory
0


In [32]:
%%capture

!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1

## 2. 데이터셋 로드

In [33]:
import glob

# 로컬 디렉토리에 저장된 TFRecord 파일들의 경로 리스트 생성
train_set_paths = glob.glob(TRAIN_SET_PATH + "/*")
valid_set_paths = glob.glob(VALID_SET_PATH + "/*")

len(train_set_paths), len(valid_set_paths)

(200, 150)

In [34]:
import tensorflow as tf

raw_train_set = tf.data.TFRecordDataset(train_set_paths)
raw_valid_set = tf.data.TFRecordDataset(valid_set_paths)

## 3. dataset parser 제작

In [207]:
from waymo_open_dataset.protos import scenario_pb2
import numpy as np

MAX_AGENT_NUM = 1000

def scenario_to_input(scenario):
  agent_num = len(scenario.tracks)
  input_data = []
  output_data = []
  for state_idx in range(len(scenario.timestamps_seconds)):
    states = []
    for track_idx in range(MAX_AGENT_NUM):
      if track_idx < agent_num:
        state = scenario.tracks[track_idx].states[state_idx]

        if state.valid:
          agent_state = [
              state.center_x, state.center_y, state.center_z,
              state.length, state.width, state.height, state.heading,
              state.velocity_x, state.velocity_y, float(state.valid)]
        else:
          agent_state = [0.0 for _ in range(10)]
      else:
        agent_state = [0.0 for _ in range(10)]
      
      states.append(agent_state)
    

    if state_idx <= scenario.current_time_index:
        input_data.append(states)
    else:
      output_data.append(states)

  return np.array([input_data]), np.array([output_data])


for raw_data in raw_train_set:
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(bytearray(raw_data.numpy()))
  
  din, dout = scenario_to_input(scenario)
  # shape : [num of state, num of agent(max), num of feature]
  print(din.shape)
  print(dout.shape)
  break

(1, 11, 1000, 10)
(1, 80, 1000, 10)


## 4. 모델 제작

In [208]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Reshape, LSTM, RepeatVector, TimeDistributed, Dense

model = Sequential()

# input
model.add(Input(shape=(11, 1000, 10)))

# cnn
model.add(Conv2D(filters=1, kernel_size=(1, 501)))
model.add(Conv2D(filters=1, kernel_size=(1, 201)))
model.add(Reshape((11, 300)))

# LSTM (many to many)
model.add(LSTM(100, activation='relu')) # encoder layer
model.add(RepeatVector(80))   # repeat vector
model.add(LSTM(100, activation='relu', return_sequences=True)) # decoder layer
model.add(TimeDistributed(Dense(10000)))

# output
model.add(Reshape((80, 1000, 10)))


model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_85"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_152 (Conv2D)         (None, 11, 500, 1)        5011      
                                                                 
 conv2d_153 (Conv2D)         (None, 11, 300, 1)        202       
                                                                 
 reshape_46 (Reshape)        (None, 11, 300)           0         
                                                                 
 lstm_192 (LSTM)             (None, 100)               160400    
                                                                 
 repeat_vector_15 (RepeatVec  (None, 80, 100)          0         
 tor)                                                            
                                                                 
 lstm_193 (LSTM)             (None, 80, 100)           80400     
                                                     

In [212]:
for idx, raw_data in enumerate(raw_train_set):
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(bytearray(raw_data.numpy()))
  _input, _output = scenario_to_input(scenario)
  print(_input.shape)
  _pred = model.predict(_input)
  print(_pred.shape)
  break

(1, 11, 1000, 10)
1/1 [==============================] - 0s 96ms/step
(1, 80, 1000, 10)


# 기타

In [ ]:
# tutorial/6_simple-model/gp-simple-model.ipynb